- date : 2024-01-29
- info : 15개의 BPIC15_1 anomaly pattern 데이터셋 만드는 파일
- BPIC15_1 label 추가해줌

In [1]:
import os
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import time
from datetime import datetime as dt
import datetime
import numpy as np

org_path = os.getcwd()

input_path = os.sep.join([str(org_path), "input"])
temp_path =  os.sep.join([str(org_path), "temp"])
output_path =  os.sep.join([str(org_path), "output"])
seed =1234

In [23]:
import importlib

from numpy import average
import abnormal_patterns
importlib.reload(abnormal_patterns)
from abnormal_patterns import Abnorm_p


# Show a list of data in 'input' folder
file_list = os.listdir(input_path)
csv_list = [s for s in file_list if 'bpic2012_O_DECLINED' in s]  # change '.csv'
print(csv_list)

['bpic2012_O_DECLINED-COMPLETE_clean.csv']


In [24]:
import importlib

from numpy import average
import abnormal_patterns
importlib.reload(abnormal_patterns)
from abnormal_patterns import Abnorm_p


# Show a list of data in 'input' folder
print(csv_list)

for dat in csv_list:
    # [Page for data import]
    print(dat)
    event_log = pd.read_csv(input_path+ '\\' + dat)

    # if dat == 'BPIC12.csv':
    #     extracted_data = event_log[['Case', 'Activity', 'Timestamp']]
    #     form = "%Y/%m/%d %H:%M:%S.%f"
    
    event_log = event_log.rename(columns={'Case ID':'Case', 'Activity':'Activity', 'Complete Timestamp':'Timestamp'})
    extracted_data = event_log[['Case', 'Activity', 'Timestamp', 'label']]
    form = "%Y-%m-%d %H:%M:%S"
    
    extracted_data.columns = ["Case", "Activity", "Timestamp", "label"] 
    extracted_data["Event"] = list(range(0,len(event_log.index)))
    cols = extracted_data.columns.tolist() 
    cols = cols[:1]+ cols[-1:] + cols[1:4] # Reorder columns
    extracted_data = extracted_data[cols]

    extracted_data = extracted_data.sort_values(["Case", "Timestamp", "Activity", "label"],ascending=[True, True, True, True]) # Reorder rows
    extracted_data.to_csv(input_path + "\\" + dat , mode='w', index=False)

['bpic2012_O_DECLINED-COMPLETE_clean.csv']
bpic2012_O_DECLINED-COMPLETE_clean.csv


In [25]:
import importlib

from numpy import average
import abnormal_patterns
importlib.reload(abnormal_patterns)
from abnormal_patterns import Abnorm_p


# Show a list of data in 'input' folder
# file_list = os.listdir(input_path)
# csv_list = [s for s in file_list if 'BPIC15_1.csv' in s]
# print(csv_list)

typearray = ["skip",  "insert", "rework", "moved", "replace"]
# rate = [0.003, 0.006, 0.009] # resource failure rate
rate = [0.02]

for typearra in typearray:
    for rat in rate:
        for dat in csv_list:
            # [Page for data import]
            print(dat)
            event_log = pd.read_csv(input_path+ '\\' + dat)

            # if dat == 'BPIC12.csv':
            #     extracted_data = event_log[['Case', 'Activity', 'Timestamp']]
            #     form = "%Y/%m/%d %H:%M:%S.%f"

            event_log = event_log.rename(columns={'Case ID':'Case', 'Activity':'Activity', 'time:timestamp':'Timestamp'})
            extracted_data = event_log[['Case', 'Activity', 'Timestamp', 'label']]
            form = "%Y-%m-%d %H:%M:%S"
            
            extracted_data.columns = ["Case", "Activity", "Timestamp", "label"] 
            extracted_data["Event"] = list(range(0,len(event_log.index)))
            cols = extracted_data.columns.tolist() 
            cols = cols[:1]+ cols[-1:] + cols[1:4] # Reorder columns
            extracted_data = extracted_data[cols]

            extracted_data = extracted_data.sort_values(["Case", "Timestamp", "Activity", "label"],ascending=[True, True, True, True]) # Reorder rows
            extracted_data['Timestamp'] = extracted_data['Timestamp'].str.slice(stop=19)
            time = extracted_data['Timestamp'].apply(lambda x: dt.strptime(x, form))
            unixtime = time.apply(lambda x: (x - dt(1970, 1, 1)).total_seconds())
            extracted_data['Timestamp'] = time
            extracted_data['unixtime'] = unixtime
            extracted_data = extracted_data.dropna(subset=['Case'])
            
            ## [Page to set root-causes of resources]

            # Just to show information of activity
            params = {
                'Case': 'count'
            }
            activitylist = extracted_data.groupby('Activity').agg(params).reset_index()

            # First to generate resource groups
            ngroup = 10  # parameter

            k = 0
            rl=list(np.repeat("act",len(activitylist)))
            for i in activitylist['Activity']:
                k += 1
                rl[k-1] = list(["Resource_Group" + str(np.random.randint(0, int(ngroup) ))])
                
            activitylist['Resource_Group'] = pd.DataFrame(rl)
            activitylist = activitylist[['Activity' , 'Resource_Group']]
            extracted_data4 = pd.merge(extracted_data, activitylist, on="Activity")

            # set a size of each group
            groupsize = 10 # parameter
            d = {'Resource_Group': ["Resource_Group" + str(i) for i in range(0, ngroup)],'Resource_Group_Size': range(0, groupsize)}
            group_size = pd.DataFrame(data=d)
            extracted_data4 = pd.merge(extracted_data4, group_size , on="Resource_Group")

            # generate resource
            attach = ["res_" + str(np.random.randint(0,int(i)+1)) for i in extracted_data4["Resource_Group_Size"]]
            extracted_data4["attach"] = attach
            extracted_data4["Resource"] = extracted_data4[['Resource_Group', 'attach']].apply(lambda x: "_".join(x) ,axis=1)
            del extracted_data4['attach']
            del extracted_data4['Resource_Group_Size']

            # extract resource-perspective data
            params = {
                'Case': 'count',
                'Activity': lambda x: ','.join(sorted(pd.Series.unique(x)))
            }

            extracted_data2 = extracted_data4
            resourcelist2 = extracted_data4.groupby('Resource').agg(params).reset_index()
            resourcelist2.columns = ["Resource", "Frequency", "Activities"]
            resourcelist2 = resourcelist2.sort_values(["Frequency"], ascending=False)
            cols = ["Resource", "Frequency",  "Activities"]
            resourcelist2 = resourcelist2[cols]
            resourcelist = resourcelist2

            # set probability of resource failure
            resourcelist2["Resource_failure_rate"] = rat  # parameter
            resourcelist3 = resourcelist2[["Resource", "Resource_failure_rate"]]
            extracted_data2 = pd.merge(extracted_data2, resourcelist3, on="Resource")

            # simulate resource failure
            np.random.seed(seed)
            PF = np.random.binomial(np.repeat(1, len(extracted_data2)), extracted_data2["Resource_failure_rate"])
            extracted_data2['Resource_Anomaly/Normal'] = PF
            extracted_data3 = extracted_data2
            

            # if 'BPIC12.csv' in dat:
            #     anomaly = Abnorm_p(extracted_data2).implement_resource(
            #         # types=["skip",  "insert", "rework", "moved", "replace"],
            #         types=[typearra],
            #         mag=[1])  # extracted_data2 = event log with pass/fail(resource)
            # types= anomaly patterns, mag= weights
                
            anomaly = Abnorm_p(extracted_data2).implement_resource(
                types=[typearra],
                mag=[1]) 
            
            # anomaly.to_csv(output_path + "\\anomaly_" +str(rate) + "_" + dat , mode='w', index=False)
            
            anomaly.to_csv(output_path + "\\"+ str(typearra) + str(rat) + "_" + dat, mode='w', index=False)
            

bpic2012_O_DECLINED-COMPLETE_clean.csv
Started preprocessing to set the input of anomaly patterns
Finished preprocessing (running time=0:00:03.424651)
Started to inject anomaly patterns
Finished to inject anomaly patterns (running time=0:00:01.253756)
bpic2012_O_DECLINED-COMPLETE_clean.csv
Started preprocessing to set the input of anomaly patterns
Finished preprocessing (running time=0:00:03.310296)
Started to inject anomaly patterns
Finished to inject anomaly patterns (running time=0:00:01.477404)
bpic2012_O_DECLINED-COMPLETE_clean.csv
Started preprocessing to set the input of anomaly patterns
Finished preprocessing (running time=0:00:03.285085)
Started to inject anomaly patterns
Finished to inject anomaly patterns (running time=0:00:01.272976)
bpic2012_O_DECLINED-COMPLETE_clean.csv
Started preprocessing to set the input of anomaly patterns
Finished preprocessing (running time=0:00:04.962374)
Started to inject anomaly patterns
Finished to inject anomaly patterns (running time=0:00:02.

In [15]:
# import os
# os.chmod('c:\\Users\\yeon1\\OneDrive - UNIST\\바탕 화면\\processed_benchmark_event_logs\\temp\\data_with_parameter1.csv', 0o700)

In [16]:
df = pd.read_csv('output/insert0.03_bpic2012_O_ACCEPTED-COMPLETE_clean.csv')
df['Case'].unique().size

4685

In [17]:
# anomaly
df.groupby("Case").filter(lambda x: x["Event"].nunique() != x["Event"].count()).groupby("Case").size()

Case
173688    27
173694    61
173715    25
173718    79
173730    79
          ..
214148    21
214220    31
214226    41
214250    25
214268    40
Length: 3118, dtype: int64

In [18]:
# normal
df.groupby("Case").filter(lambda x: x["Event"].nunique() == x["Event"].count()).groupby("Case").size()

Case
173691    39
173721    35
173736    55
173760    26
173787    35
          ..
214058    31
214061    40
214238    30
214277    28
214361    28
Length: 1567, dtype: int64